## Amazon AgentCore Bedrock Code Interpreter Tutorial

This tutorial demonstrates how to use AgentCore Bedrock Code Interpreter to:
1. Set up a sandbox environment
2. Load and analyze data
3. Execute code in a sandbox environment
4. Process and retrieve results

## Prerequisites
- AWS account with Bedrock AgentCore Code Interpreter access
- You have the necessary IAM permissions to create and manage code interpreter resources
- Required Python packages installed(including boto3 & bedrock-agentcore)
- Sample data file (data.csv)
- Analysis script (stats.py)


## Your IAM execution role should have the following IAM policy attached

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}



## How it works

The code execution sandbox enables agents to safely process user queries by creating an isolated environment with a code interpreter, shell, and file system. After a Large Language Model helps with tool selection, code is executed within this session, before being returned to the user or agent for synthesis.

![architecture local](code-interpreter.png)

## 1. Setting Up the Environment

First, let's import the necessary libraries and initialize our Code Interpreter client.

In [ ]:
!pip install --upgrade -r requirements.txt

In [21]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import json
from typing import Dict, Any, List

# Initialize the Code Interpreter with your AWS region
code_client = CodeInterpreter('us-west-2')
code_client.start()

'01K00Z3F8WZ9KBBW4QGRJCVBHH'

## 2. Reading Local Files

Now we'll read the contents of our sample data file and analysis script.

In [22]:
def read_file(file_path: str) -> str:
    """Helper function to read file content with error handling"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

# Read both files
data_file_content = read_file("samples/data.csv")
code_file_content = read_file("samples/stats.py")

## 3. Preparing Files for Sandbox Environment

We'll create a structure that defines the files we want to create in the sandbox environment.

In [23]:
files_to_create = [
    {
        "path": "data.csv",
        "text": data_file_content
    },
    {
        "path": "stats.py",
        "text": code_file_content
    }
]

## 4. Creating Helper Function for Tool Invocation

This helper function will make it easier to call sandbox tools and handle their responses. Within an active session, you can execute code in supported languages (Python, JavaScript), access libraries based on your dependencies configuration, generate visualizations, and maintain state between executions.

In [24]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to invoke sandbox tools
    
    Args:
        tool_name (str): Name of the tool to invoke
        arguments (Dict[str, Any]): Arguments to pass to the tool
        
    Returns:
        Dict[str, Any]: JSON formatted result
    """
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. Writing Files to Sandbox

Now we'll write our files into the sandbox environment and verify they were created successfully.

In [25]:
# Write files to sandbox
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("Writing files result:")
print(writing_files)

# Verify files were created
listing_files = call_tool("listFiles", {"path": ""})
print("\nFiles in sandbox:")
print(listing_files)

Writing files result:
{"content": [{"type": "text", "text": "Successfully wrote all 2 files"}], "isError": false}

Files in sandbox:
{"content": [{"type": "resource_link", "uri": "file:///log", "name": "log", "description": "Directory"}, {"type": "resource_link", "mimeType": "text/csv", "uri": "file:///data.csv", "name": "data.csv", "description": "File"}, {"type": "resource_link", "mimeType": "text/x-python", "uri": "file:///stats.py", "name": "stats.py", "description": "File"}, {"type": "resource_link", "uri": "file:///.ipython", "name": ".ipython", "description": "Directory"}], "isError": false}


## 6. Executing Analysis

Now we'll execute our analysis script in the sandbox environment and process the results.

In [26]:
import pprint

# Execute the analysis script
code_execute_result = call_tool("executeCode", {
    "code": files_to_create[1]['text'],
    "language": "python",
    "clearContext": True
})

# Parse and display results
analysis_results = json.loads(code_execute_result)
print("Full analysis results:")
pprint.pprint(analysis_results)

print("\nStandard output from analysis:")
print(analysis_results['structuredContent']['stdout'])

Full analysis results:
{'content': [{'text': 'Name   Place  Animal   Thing\n'
                      'count       299130  299130  299130  299130\n'
                      'unique        1722      55      50      51\n'
                      'top     Lisa White  Prague    Goat  Pencil\n'
                      'freq           222    5587    6141    6058',
              'type': 'text'}],
 'isError': False,
 'structuredContent': {'executionTime': 0.7111423015594482,
                       'exitCode': 0,
                       'stderr': '',
                       'stdout': 'Name   Place  Animal   Thing\n'
                                 'count       299130  299130  299130  299130\n'
                                 'unique        1722      55      50      51\n'
                                 'top     Lisa White  Prague    Goat  Pencil\n'
                                 'freq           222    5587    6141    6058'}}

Standard output from analysis:
Name   Place  Animal   Thing
count       29

## 7. Cleanup

Finally, we'll clean up by stopping the Code Interpreter session. Once finished using a session, the session should be shopped to release resources and avoid unnecessary charges.

In [27]:
# Stop the Code Interpreter session
code_client.stop()
print("Code Interpreter session stopped successfully!")

Code Interpreter session stopped successfully!


## Conclusion

In this tutorial, we've learned how to:
- Initialize a Code Interpreter session
- Read and prepare files for analysis
- Execute code in a sandbox environment
- Process and display results
- Clean up resources